[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Denis-R-V/Homeworks/blob/main/Part_3/Task_4_ML2.ipynb)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import optuna
import plotly.express as px

https://archive.ics.uci.edu/ml/datasets/wine+quality

In [2]:
# данные можно взять с сайта
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(link, sep = ';')

In [3]:
# загразка датасета из локального файла
data = pd.read_csv('winequality-red.csv', sep = ';')
data.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5


# 1.Разделите выборку на обучающую и тестовую в отношении 70%/30%

Разделим выборку на признаки и целевую переменную

In [4]:
X = data.iloc[:,:11].copy()
X.head(3)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8


In [5]:
y = data['quality'].copy()
y.head(3)

0    5
1    5
2    5
Name: quality, dtype: int64

XGBoost ругается на целевую переменную:  
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [3 4 5 6 7 8]  
Для устранения ошибки можно кодировать целевую переменную с помощью LabelEncoder

In [6]:
y_encoder = LabelEncoder()
# преобразованный y_train и y_test
y_enc = y_encoder.fit_transform(y)

In [7]:
# Обратное преобразование для получение исходных целевых переменных после предсказания
y_encoder.inverse_transform(y_enc)

array([5, 5, 5, ..., 6, 5, 6], dtype=int64)

In [8]:
# элементы целевой переменной до преобразования и после прямого и обратного преобразований равны
(y == y_encoder.inverse_transform(y_enc)).unique()

array([ True])

Разделим выборку на обучающую и тестовую

In [9]:
# для повторяемости результата зафиксируем randome state
RANDOM_STATE = 42

In [10]:
# Разбиваем выборку на обучающую и тестовую с помьщью функции train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_enc, train_size=0.7, random_state=RANDOM_STATE)  # добавляем random stete для повторяемости результата

# Выводим размеры тестовой и обучающей выборок и целевой переменной
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1119, 11), (480, 11), (1119,), (480,))

# 2.Оцените качество по метрике accuracy для классификаторов:
- DecisionTreeClassifier
- BaggingClassifier со 100 деревьями
- RandomForestClassifier со 100 деревьями
## Сравните результаты и напишите какой вывод можно сделать.

DecisionTreeClassifier

In [11]:
DecTreeClas = DecisionTreeClassifier(random_state=RANDOM_STATE)
DecTreeClas.fit(X_train, y_train)
DecTreeClas_accaracy = accuracy_score(y_test, DecTreeClas.predict(X_test))
print(f"Accuracy DecisionTreeClassifier: {DecTreeClas_accaracy:.6f}")

Accuracy DecisionTreeClassifier: 0.572917


BaggingClassifier со 100 деревьями

In [12]:
BaggClas = BaggingClassifier(n_estimators=100, random_state=RANDOM_STATE)
BaggClas.fit(X_train, y_train)
BaggClas_accaracy = accuracy_score(y_test, BaggClas.predict(X_test))
print(f"Accuracy BaggingClassifier: {BaggClas_accaracy:.6f}")

Accuracy BaggingClassifier: 0.641667


RandomForestClassifier со 100 деревьями

In [13]:
RandFrstClas = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
RandFrstClas.fit(X_train, y_train)
RandFrstClas_accaracy = accuracy_score(y_test, RandFrstClas.predict(X_test))
print(f"Accuracy RandomForestClassifier: {RandFrstClas_accaracy:.6f}")

Accuracy RandomForestClassifier: 0.654167


Вывод: ансамблевые методы классификации (BaggingClassifier и RandomForestClassifier) отработали значительно лучше, чем простое дерево решений.

# 3.Посчитайте качество на тестовой выборке по метрике accuracy для классификатора RandomForestClassifier, используя значения деревьев: 10, 50, 100, 200, далее с шагом 200 до 5000 деревьев. Постройте график зависимости качества от числа деревьев.

Список количества деревьев для цикла

In [14]:
n_estimators = [10, 50, 100] + list(range(200, 5001, 200))
print(n_estimators)

[10, 50, 100, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000, 4200, 4400, 4600, 4800, 5000]


Обучение моделей по заданному количеству деревьев из списка и расчет метрик accuracy на тестовых данных

In [15]:
RandFrstClas_estim_accuracy = {}
for i in n_estimators:
    RandFrstClas_temp = RandomForestClassifier(n_estimators=i, random_state=RANDOM_STATE)
    RandFrstClas_temp.fit(X_train, y_train)
    RandFrstClas_temp_accaracy = accuracy_score(y_test, RandFrstClas_temp.predict(X_test))
    RandFrstClas_estim_accuracy[i] = RandFrstClas_temp_accaracy
RandFrstClas_estim_accuracy

{10: 0.6333333333333333,
 50: 0.6520833333333333,
 100: 0.6541666666666667,
 200: 0.6645833333333333,
 400: 0.6625,
 600: 0.6645833333333333,
 800: 0.6645833333333333,
 1000: 0.6645833333333333,
 1200: 0.6604166666666667,
 1400: 0.6645833333333333,
 1600: 0.6604166666666667,
 1800: 0.6625,
 2000: 0.66875,
 2200: 0.66875,
 2400: 0.66875,
 2600: 0.6666666666666666,
 2800: 0.66875,
 3000: 0.6708333333333333,
 3200: 0.66875,
 3400: 0.6645833333333333,
 3600: 0.6645833333333333,
 3800: 0.66875,
 4000: 0.66875,
 4200: 0.6708333333333333,
 4400: 0.6666666666666666,
 4600: 0.6645833333333333,
 4800: 0.6666666666666666,
 5000: 0.66875}

Визуализация результатов

In [16]:
fig = px.line(pd.DataFrame(data=[RandFrstClas_estim_accuracy]).T.rename(columns={0:'accuracy'}),
              labels={'index':'Количество деревьев', 'value':'Значение метрики', 'variable':'Метрика'},
              title="Зависимость качества классификатора RandomForestClassifier от количества деревьев",
              color_discrete_sequence=['red'],
              markers=True)
fig.show()

Выводы:
- при увеличении количества деревьев до 200 качество классификатора RandomForestClassifier растет;
- дальнейшее увеличение количества деревьев свыше 200 качество классификатора RandomForestClassifier существенно не повышает.

# 4.Обучите реализации градиентного бустинга с параметрами по умолчанию из библиотек sklearn и xgboost. Сравните значение метрики accuracy по cross_val_score

Градиентный бустинг с параметрами по умолчанию из библиотеки sklearn

In [17]:
GradBst = GradientBoostingClassifier(random_state=RANDOM_STATE)
GradBst_accuracy = cross_val_score(GradBst, X, y_enc, cv=5, scoring='accuracy').mean()
print(f"accuracy GradientBoostingClassifier cross_val_score: {GradBst_accuracy:.4f}")

accuracy GradientBoostingClassifier cross_val_score: 0.5653


Градиентный бустинг с параметрами по умолчанию из библиотеки xgboost

In [18]:
XGBClas = XGBClassifier(random_state=RANDOM_STATE)
XGBClas_accuracy = cross_val_score(XGBClas, X, y_enc, cv=5, scoring='accuracy').mean()
print(f"accuracy XGBClassifier cross_val_score: {XGBClas_accuracy:.4f}")

accuracy XGBClassifier cross_val_score: 0.5472


Вывод: с параметрами по умолчанию у GradientBoostingClassifier значение метрики accuracy несколько лучше, чем у XGBClassifier, но скорость работы в 3.5 раза дольше

# 5.Подберите оптимальные параметры этих алгоритмов с помощью GridSearchCV(cv=3).
Параметры для оптимизации:
- скорость обучения
- количество деревьев
- глубина деревьев  
Сравните значение метрики accuracy и скорость работы. Выведите лучшие параметры алгоритмов.

Параметры для оптимизации

In [30]:
parameters = {'learning_rate': [0.0001, 0.001, 0.01, 0.03, 0.06, 0.1, 0.2, 0.3], 
              'n_estimators':[1, 5, 10, 100, 200, 300, 400, 500, 600, 700],
              'max_depth': np.arange(1,7,1)
              }

Подбор оптимальных параметров для GradientBoostingClassifier с помощью GridSearchCV

In [20]:
GradBst_best = GridSearchCV(GradientBoostingClassifier(random_state=RANDOM_STATE), cv=3,
                            param_grid=parameters,
                            scoring=('accuracy'))
GradBst_best.fit(X_train, y_train)
print("Подобранные параметры для GradientBoostingClassifier:")
display(GradBst_best.best_params_)

Подобранные параметры для GradientBoostingClassifier:


{'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100}

Подбор оптимальных параметров для XGBClassifier с помощью GridSearchCV

In [21]:
XGBClas_best = GridSearchCV(XGBClassifier(random_state=RANDOM_STATE), cv=3,
                            param_grid=parameters,
                            scoring=('accuracy'))
XGBClas_best.fit(X_train, y_train)

print("Подобранные параметры для XGBClassifier:")
display(XGBClas_best.best_params_)

Подобранные параметры для XGBClassifier:


{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 300}

Сравнение метрик на тестовых данных

In [22]:
GradBst_best_accuracy = accuracy_score(y_test, GradBst_best.predict(X_test))
XGBClas_best_accuracy = accuracy_score(y_test, XGBClas_best.predict(X_test))
print(f"accuracy GradientBoostingClassifier best model test: {GradBst_best_accuracy:.6f}")
print(f"accuracy XGBClassifier best model test:              {XGBClas_best_accuracy:.6f}")

accuracy GradientBoostingClassifier best model test: 0.641667
accuracy XGBClassifier best model test:              0.650000


Выводы:
- с подобранными параметрами на тестовых данных значение метрики для XGBClassifier несколько лучше, чем для GradientBoostingClassifier,
- при этом подбор параметров для XGBClassifier отработал в 5 раз быстрее (в моем случае на рассматриваемых данных)

# 6.Обучите реализации градиентного бустинга с параметрами по умолчанию из библиотек lightgbm и catboost. Сравните значение метрики accuracy по cross_val_score по всем четырем реализациям.

Градиентный бустинг с параметрами по умолчанию из библиотеки lightgbm

In [23]:
LiGBM = LGBMClassifier(random_state=RANDOM_STATE)
LiGBM_accuracy = cross_val_score(LiGBM, X, y_enc, cv=5, scoring='accuracy').mean()
print(f"accuracy LGBMClassifier cross_val_score: {LiGBM_accuracy:.4f}")

accuracy LGBMClassifier cross_val_score: 0.5572


Градиентный бустинг с параметрами по умолчанию из библиотеки catboost

In [24]:
CatBst = CatBoostClassifier(random_state=RANDOM_STATE, logging_level='Silent')
CatBst_accuracy = cross_val_score(CatBst, X, y_enc, cv=5, scoring='accuracy').mean()
print(f"accuracy CatBoostClassifier cross_val_score: {CatBst_accuracy:.4f}")

accuracy CatBoostClassifier cross_val_score: 0.5516


In [25]:
print("Сравнение значений метрики accuracy для градиентного бустинга из разных библиотек (параметры по умолчанию):")
print(f"accuracy GradientBoostingClassifier cross_val_score: {GradBst_accuracy:.4f}")
print(f"accuracy XGBClassifier cross_val_score:              {XGBClas_accuracy:.4f}")
print(f"accuracy LGBMClassifier cross_val_score:             {LiGBM_accuracy:.4f}")
print(f"accuracy CatBoostClassifier cross_val_score:         {CatBst_accuracy:.4f}")

Сравнение значений метрики accuracy для градиентного бустинга из разных библиотек (параметры по умолчанию):
accuracy GradientBoostingClassifier cross_val_score: 0.5653
accuracy XGBClassifier cross_val_score:              0.5472
accuracy LGBMClassifier cross_val_score:             0.5572
accuracy CatBoostClassifier cross_val_score:         0.5516


Выводы:
- с параметрами по умолчанию наилучшее значение метрики accuracy у GradientBoostingClassifier;
- CatBoostClassifier отработал сравнительно долго и показал не самую лучшую метрику.

# 7.Подберите оптимальные параметры для алгоритмов градиентного бустинга из библиотек lightgbm и catboost с теми же условиями. Выведите лучшие параметры алгоритмов. Сравните значение метрики accuracy и скорость по этим четырем реализациям.

Подбор оптимальных параметров для LGBMClassifier с помощью GridSearchCV

In [26]:
LiGBM_best = GridSearchCV(LGBMClassifier(random_state=RANDOM_STATE), cv=3,
                            param_grid=parameters,
                            scoring=('accuracy'))
LiGBM_best.fit(X_train, y_train)
print("Подобранные параметры для LGBMClassifier:")
display(LiGBM_best.best_params_)

Подобранные параметры для LGBMClassifier:


{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 300}

Подбор оптимальных параметров для CatBoostClassifier с помощью GridSearchCV

In [27]:
CatBst_best = GridSearchCV(CatBoostClassifier(random_state=RANDOM_STATE, logging_level='Silent'), cv=3,
                           param_grid=parameters,
                           scoring=('accuracy')
                           )
CatBst_best.fit(X_train, y_train)
print("Подобранные параметры для CatBoostClassifier:")
display(CatBst_best.best_params_)

Подобранные параметры для CatBoostClassifier:


{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 300}

Сравнение метрик алгоритмов с оптимальными параметрами на тестовых данных

In [28]:
LiGBM_best_accuracy = accuracy_score(y_test, LiGBM_best.predict(X_test))
CatBst_best_accuracy = accuracy_score(y_test, CatBst_best.predict(X_test))
print("Сравнение значений метрики accuracy для градиентного бустинга из разных библиотек (параметры подобраны с помощью GridSearchCV):")
print(f"accuracy GradientBoostingClassifier best model test: {GradBst_best_accuracy:.6f}")
print(f"accuracy XGBClassifier best model test:              {XGBClas_best_accuracy:.6f}")
print(f"accuracy LGBMClassifier best model test:             {LiGBM_best_accuracy:.6f}")
print(f"accuracy CatBoostClassifier best model test:         {CatBst_best_accuracy:.6f}")

Сравнение значений метрики accuracy для градиентного бустинга из разных библиотек (параметры подобраны с помощью GridSearchCV):
accuracy GradientBoostingClassifier best model test: 0.641667
accuracy XGBClassifier best model test:              0.650000
accuracy LGBMClassifier best model test:             0.662500
accuracy CatBoostClassifier best model test:         0.641667


In [45]:
# сведем результаты в таблицу
result = pd.DataFrame(data=[[GradBst_accuracy, GradBst_best_accuracy, '54m 56.6s'],
                            [XGBClas_accuracy, XGBClas_best_accuracy, '12m 26.7s'],
                            [LiGBM_accuracy, LiGBM_best_accuracy, '6m 58.6s'],
                            [CatBst_accuracy, CatBst_best_accuracy, '11m 23.1s']],
                      index=['GradientBoostingClassifier','XGBClassifier','LGBMClassifier','CatBoostClassifier'],
                      columns = pd.MultiIndex.from_tuples([('Default', 'accuracy'), ('GridSearchCV', 'accuracy'), ('GridSearchCV', 'time')],
                                                             names=['Parameters', 'Values']))
result

Parameters                   Default GridSearchCV           
Values                      accuracy     accuracy       time
GradientBoostingClassifier  0.565347     0.641667  54m 56.6s
XGBClassifier               0.547234     0.650000  12m 26.7s
LGBMClassifier              0.557247     0.662500   6m 58.6s
CatBoostClassifier          0.551611     0.641667  11m 23.1s

Выводы:
- с подобранными параметрами значение метрик существенно выше для всех алгоритмов, чем с параметрами по умолчанию;
- с подобранными параметрами с помощью GridSearchCV наилучшее значение метрики accuracy у алгоритма градиентного бустинга из библиотеки LGBMClassifier;
- при этом скорость обучения (подбора параметров) у алгоритма градиентного бустинга из библиотеки LGBMClassifier в разы ниже, чем у алгоритмов из остальных библиотек.

# 8.Подберите оптимальные параметры алгоритма из библиотеки xgbost с помощью [optuna](https://github.com/optuna/optuna).
Параметры для оптимизации:
- скорость обучения
- количество деревьев
- глубина деревьев  
Сравните результат с поиском по сетке из sklearn. Выведите лучшие параметры алгоритма, найденные даным способом.

Подбор параметров с cross_val_score без участия в подборе тестовой выборки

In [37]:
def opt(trial):
    # границы оптимизируемых параметров
    n_estimators = trial.suggest_int('n_estimators', 0, 1000)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    learning_rate = trial.suggest_float('learning_rate', 0, 1.0)
    
    # обучаемая модель
    xgboost_tuna = XGBClassifier(random_state=42, 
                                 tree_method='gpu_hist',
                                 n_estimators = n_estimators,
                                 max_depth = max_depth,
                                 learning_rate = learning_rate,
                                 )
    return cross_val_score(xgboost_tuna, X_train, y_train, cv=3, scoring='accuracy').mean()
study = optuna.create_study(direction="maximize")   # максимизируем метрику accuracy
study.optimize(opt, n_trials=500)                   # количество прогонов оптимизации

[I 2022-11-06 23:00:07,331] A new study created in memory with name: no-name-2a51a497-f673-4e06-b0e9-223a0344f297
[I 2022-11-06 23:00:24,155] Trial 0 finished with value: 0.6273458445040214 and parameters: {'n_estimators': 337, 'max_depth': 6, 'learning_rate': 0.07334688539626455}. Best is trial 0 with value: 0.6273458445040214.
[I 2022-11-06 23:00:33,765] Trial 1 finished with value: 0.6264521894548704 and parameters: {'n_estimators': 151, 'max_depth': 9, 'learning_rate': 0.11856677290580031}. Best is trial 0 with value: 0.6273458445040214.
[I 2022-11-06 23:00:42,928] Trial 2 finished with value: 0.5987488829311886 and parameters: {'n_estimators': 286, 'max_depth': 9, 'learning_rate': 0.9413202408189493}. Best is trial 0 with value: 0.6273458445040214.
[I 2022-11-06 23:00:47,295] Trial 3 finished with value: 0.6193029490616622 and parameters: {'n_estimators': 73, 'max_depth': 16, 'learning_rate': 0.4401355357815374}. Best is trial 0 with value: 0.6273458445040214.
[I 2022-11-06 23:01:

# Расчет метрики XGBClassifier с параметрами подобранными с помощью optuna и сравнение с GridSearchCV

In [40]:
XGBClas_optun_best = XGBClassifier(tree_method='gpu_hist',**study.best_params)
XGBClas_optun_best.fit(X_train, y_train)
XGBClas_optun_best_accuracy = accuracy_score(y_test, XGBClas_optun_best.predict(X_test))
print(f"Подобранные параметры для XGBClassifier с помощью optuna:")
display(study.best_params)
print(f"accuracy XGBClassifier best model with GridSearchCV test: {XGBClas_best_accuracy:.6f}")
print(f"accuracy XGBClassifier best model with optuna test:       {XGBClas_optun_best_accuracy:.6f}")

Подобранные параметры для XGBClassifier с помощью optuna:


{'n_estimators': 146, 'max_depth': 6, 'learning_rate': 0.06296398088567319}

accuracy XGBClassifier best model with GridSearchCV test: 0.650000
accuracy XGBClassifier best model with optuna test:       0.650000


Выводы:
- на тестовых данных значение метрики accutacy XGBClassifier с параметрами подобранными GridSearchCV и optuna для одинаково;
- вероятно с помощью GridSearchCV была удачно выставлена сетка параметров;
- optuna подбирает параметры в более широком диапазоне, в теории значение метрики должно быть лучше.

# 9.Выведите качество по метрике accuracy стэкинга (StackingClassifier) 4-х алгоритмов с базовыми параметрами градиентного бустинга.

In [44]:
estimators = [('GradientBoostingClassifier', GradientBoostingClassifier(random_state=RANDOM_STATE)),
              ('XGBClassifier', XGBClassifier(random_state=RANDOM_STATE)),
              ('LGBMClassifier', LGBMClassifier(random_state=RANDOM_STATE)),
              ('CatBoostClassifier', CatBoostClassifier(random_state=RANDOM_STATE, logging_level='Silent'))
              ]
Stacking_models_default = StackingClassifier(estimators=estimators)
Stacking_models_default.fit(X_train, y_train)
Stacking_models_default_accuracy = accuracy_score(y_test, Stacking_models_default.predict(X_test))
print(f"accuracy StackingClassifier models with default parameters: {Stacking_models_default_accuracy:.6f}")

accuracy StackingClassifier models with default parameters: 0.677083


Вывод: даже с параметрами по умолчанию StackingClassifier по метрике accuracy превзошел другие рассмотренные алгоритмы, в том числе с подобранными параметрами

# 10.Выведите качество по метрике accuracy стэкинга 4-х алгоритмов с оптимальными параметрами градиентного бустинга. Сравните результаты с предыдущим шагом и напишите какой вывод можно из этого сделать.

In [50]:
estimators = [('GradientBoostingClassifier', GradientBoostingClassifier(**GradBst_best.best_params_, random_state=RANDOM_STATE)),
              ('XGBClassifier', XGBClassifier(**study.best_params, random_state=RANDOM_STATE)),
              ('LGBMClassifier', LGBMClassifier(**LiGBM_best.best_params_, random_state=RANDOM_STATE)),
              ('CatBoostClassifier', CatBoostClassifier(**CatBst_best.best_params_, random_state=RANDOM_STATE, logging_level='Silent'))
              ]
Stacking_models_best = StackingClassifier(estimators=estimators)
Stacking_models_best.fit(X_train, y_train)
Stacking_models_best_accuracy = accuracy_score(y_test, Stacking_models_best.predict(X_test))
print(f"accuracy StackingClassifier models with default parameters: {Stacking_models_default_accuracy:.6f}")
print(f"accuracy StackingClassifier models with best parameters:    {Stacking_models_best_accuracy:.6f}")

accuracy StackingClassifier models with default parameters: 0.677083
accuracy StackingClassifier models with best parameters:    0.668750


In [51]:
# сведем результаты в таблицу
result = pd.DataFrame(data=[[GradBst_accuracy, GradBst_best_accuracy, '54m 56.6s'],
                            [XGBClas_accuracy, XGBClas_best_accuracy, '12m 26.7s'],
                            [LiGBM_accuracy, LiGBM_best_accuracy, '6m 58.6s'],
                            [CatBst_accuracy, CatBst_best_accuracy, '11m 23.1s'],
                            [Stacking_models_default_accuracy, Stacking_models_best_accuracy, np.NaN]],
                      index=['GradientBoostingClassifier','XGBClassifier','LGBMClassifier','CatBoostClassifier','StackingClassifier'],
                      columns = pd.MultiIndex.from_tuples([('Default', 'accuracy'), ('Optimal', 'accuracy'), ('Optimal', 'time')],
                                                             names=['Parameters', 'Values']))
result

Parameters                   Default   Optimal           
Values                      accuracy  accuracy       time
GradientBoostingClassifier  0.565347  0.641667  54m 56.6s
XGBClassifier               0.547234  0.650000  12m 26.7s
LGBMClassifier              0.557247  0.662500   6m 58.6s
CatBoostClassifier          0.551611  0.641667  11m 23.1s
StackingClassifier          0.677083  0.668750        NaN

Выводы:
- StackingClassifier на алгоритмах с подобранными параметрами отработал хуже чем на этих алгоритмах с подобранными параметрами;
- вероятно дело в том, что параметры подбираются для работы одного алгоритма;
- лучшее качество по метрике accuracy у стэкинга (StackingClassifier) 4-х алгоритмов с базовыми параметрами градиентного бустинга.